In [ ]:
# Correct H2 molecule VQE (R = 0.735 Å, STO-3G basis)
import numpy as np
import matplotlib.pyplot as plt
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import TwoLocal
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2, Session
from scipy.optimize import minimize
from qiskit.transpiler import generate_preset_pass_manager

# ---- CORRECT H2 Hamiltonian (R = 0.735 Å, should give ≈ -1.137 Ha) ----
# Standard literature values for H2 molecule at equilibrium
h2_correct_paulis = [
    ("II", -1.0636533),      # Nuclear repulsion + constant terms
    ("ZI", -0.3980107),      # Single qubit Z rotation
    ("IZ", -0.3980107),      # Single qubit Z rotation  
    ("ZZ", -0.0112801),      # Two-qubit ZZ interaction
    ("XX", 0.1809312),       # Two-qubit XX interaction
    ("YY", 0.1809312)        # Two-qubit YY interaction
]

h2_hamiltonian = SparsePauliOp.from_list(h2_correct_paulis)

# Calculate exact energies
eigenvalues, eigenvectors = np.linalg.eigh(h2_hamiltonian.to_matrix())
exact_energy = min(eigenvalues)



# ---- Ansatz Circuit ----
ansatz = TwoLocal(
    num_qubits=h2_hamiltonian.num_qubits,  # 2 qubits for H2
    rotation_blocks=["ry", "rz"],          # Single-qubit rotations
    entanglement_blocks="cz",              # Controlled-Z gates
    entanglement="linear",                 # Linear connectivity
    reps=2,                               # 2 layers
    insert_barriers=True
)

optimized_params = np.array([
    -1.10060477e+00,  3.10876865e+00,  7.24378066e-02,
     1.09902187e+00, -1.14739548e+00, -2.78898113e+00,
    -3.14208770e+00,  2.77528083e+00,  3.17333049e-03,
     3.96805627e-01, -3.01411231e+00,  4.34994209e+00
])

# ---- QPU VQE (commented out - uncomment to run on real hardware) ----
def run_qpu_vqe():
    """Run VQE on IBM Quantum hardware"""
    print("\n=== QPU VQE ===")
    
    # Initialize service
    service = QiskitRuntimeService()
    backend = service.least_busy(operational=True, simulator=False)
    print(f"Selected backend: {backend.name}")
    
    cost_history = []
    
    def cost_function_qpu(params):
        # Configure estimator options
        estimator.options.resilience_level = 1  # Error mitigation
        estimator.options.default_shots = 4096  # Number of shots
        
        pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
        isa_circuit = pm.run(ansatz)

        pub = (isa_circuit, h2_hamiltonian, params)
        job = estimator.run([pub])
        result = job.result()
        energy = result[0].data.evs
        
        cost_history.append(energy)
        print(f"  QPU Iteration {len(cost_history)}: Energy = {energy:.6f} Ha")
        
        return energy
    
    
    with Session(backend=backend, max_time="10m") as session:
        estimator = EstimatorV2(session=session)
        
        result = minimize(
            cost_function_qpu,
            optimized_params,
            method="COBYLA",
            options={"maxiter": 20, "rhobeg": 0.05}
        )
    
    print(f"\nQPU Results:")
    print(f"  Final energy: {result.fun:.6f} Ha")
    print(f"  Exact energy: {exact_energy:.6f} Ha")
    print(f"  Error: {abs(result.fun - exact_energy):.6f} Ha")
    
    return result, cost_history

result, cost_h = run_qpu_vqe()

/var/folders/d0/x8lpmss173z38_qn8c66v27h0000gn/T/ipykernel_42539/3014971355.py:31: DeprecationWarning: The class ``qiskit.circuit.library.n_local.two_local.TwoLocal`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.n_local instead.
  ansatz = TwoLocal(



=== QPU VQE ===


RequestsApiError: '403 Client Error: Forbidden for url: https://us-east.quantum-computing.cloud.ibm.com/backends. {"errors":[{"code":1200,"message":"You are not authorized to perform this action.","solution":"Try again or contact support.","more_info":"https://cloud.ibm.com/apidocs/quantum-computing#error-handling"}],"trace":"508efc13-537a-4f1e-8a84-ec337d23a3bf"}'